In [1]:
import pandas as pd
import numpy as np
import gc

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, auc, precision_score, recall_score
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, StandardScaler, RobustScaler

SEED = 1996
FOLDS = 5
TARGET = "y"
VERSION = "lgbm-v1"

# Load Data

In [2]:
df_train = pd.read_csv("../data/train.csv")
df_train.head()

,id,var1,var2,var3,var4,var5,var6,var7,var8,var9,...,var60,var61,var62,var63,var64,var65,var66,var67,var68,y
0,1,18,19,2853,29442,1386,2435,35,-999,3,...,0.311441,0.142303,0.056146,0.632694,0.024054,0.253356,0.00603,0.132353,0.139706,1
1,8,4,110,1986,13684,7189,-999,-999,17,3,...,-999.000000,-999.000000,0.070991,0.773966,0.019315,-999.000000,-999.00000,0.147059,0.106618,0
2,30,0,39,1019,10232,678,791,16,-999,3,...,-999.000000,0.200814,0.051046,0.980827,0.018536,-999.000000,-999.00000,0.382353,0.242647,0
3,43,20,39,1751,2689,8235,1042,13,10,1,...,-999.000000,0.352379,0.044301,0.951564,0.023684,0.363370,0.00201,0.147059,0.132353,0
4,46,7,44,2262,29428,6031,304,16,-999,3,...,0.021226,0.226161,0.059125,0.906155,0.020733,-999.000000,-999.00000,0.455882,0.132353,1


In [3]:
df_test = pd.read_csv("../data/test.csv")
df_test.head()

,id,var1,var2,var3,var4,var5,var6,var7,var8,var9,...,var59,var60,var61,var62,var63,var64,var65,var66,var67,var68
0,0,5,126,1353,28956,743,1289,27,-999,1,...,0.201839,0.353965,0.166641,0.049108,0.986882,0.016683,-999.0,-999.0,0.176471,0.253676
1,2,6,126,1446,7803,5151,935,35,-999,3,...,0.072127,0.074555,0.217009,0.144403,0.892028,0.038323,-999.0,-999.0,0.147059,0.099265
2,4,5,44,243,4325,1109,1903,33,24,1,...,0.324770,0.384992,0.330680,0.072864,0.930373,0.021052,-999.0,-999.0,0.294118,0.136029
3,7,4,53,419,743,7750,183,35,-999,3,...,0.131070,-999.000000,0.244936,0.158088,0.986882,0.022649,-999.0,-999.0,0.294118,0.220588
4,15,4,126,1863,22693,5625,965,9,-999,3,...,0.225166,0.059940,0.252794,0.080405,0.944501,0.021806,-999.0,-999.0,0.352941,0.113971


In [4]:
df_sub = pd.read_csv("../data/submission_sample.csv")
df_sub.head()

,id,predicted
0,0,1
1,2,1
2,4,1
3,7,0
4,15,1


In [5]:
df_meta = pd.read_csv("../data/metadata.csv")
df_meta.head()

,Variavel cod,Variavel tipo
0,id,Qualitativo nominal
1,var1,Qualitativo nominal
2,var2,Qualitativo nominal
3,var3,Qualitativo nominal
4,var4,Qualitativo nominal


In [6]:
df_meta["Variavel tipo"].value_counts()

Qualitativo nominal      36
Quantitativo discreto    18
Quantitativo continua    12
Qualitativo ordinal       4
Name: Variavel tipo, dtype: int64

## Vars type

- **Variável qualitativa nominal** = valores que expressam atributos, sem
nenhum tipo de ordem. Ex: cor dos olhos, sexo, estado civil, presença ou
ausência...


- **Variável qualitativa ordinal** = valores que expressam atributos, porém com
algum tipo de ordem, ou grau. Ex: grau de escolaridade (1º grau, 2º grau, 3º
grau, pós-graduação...); resposta de um paciente (nenhuma melhora, alguma
melhora, muita melhora); classe social (alta, média, baixa)...


- **Variável quantitativa discreta** = valores observados somente em pontos
isolados ao longo de uma escala de valores (contagem). Valores positivos
inteiros (incluindo o zero). Ex: No de filhos; No de faltas; alunos com notas abaixo de 5,0.


- **Variável quantitativa contínua** = valores em qualquer ponto fracionário ao
longo de um intervalo especificado de valores (medição). Ex: temperatura do
corpo; altura (em metros); índice do PIB...

In [7]:
cols_type = df_meta.drop(0).groupby("Variavel tipo")["Variavel cod"].apply(list)
cols_type

Variavel tipo
Qualitativo nominal      [var1, var2, var3, var4, var5, var6, var7, var...
Qualitativo ordinal                           [var26, var32, var42, var43]
Quantitativo continua    [var55, var56, var57, var58, var59, var60, var...
Quantitativo discreto    [var24, var25, var27, var40, var44, var45, var...
Name: Variavel cod, dtype: object

# Features

## Features ideas

There are many approaches to feature engineering, which could be also used on anonymized data:

- transformation of continuous variables: log, power, normalization;
- aggregations: `df.groupby(['cat_column']).agg({'continuous_column': ['min', 'max', 'mean', 'std'])
- interactions of continuous variables: addition, subtraction, multiplications, division And so on.

## Dummies

In [8]:
# cols = cols_type[0]
# cols.extend(cols_type[1])

# print("shape before dummies: ", df_train.shape)

# for col in cols:
#     df_dummies = pd.get_dummies(df_train[col])
#     df_dummies.columns = [f"{col}_{suffix}" for suffix in df_dummies.columns]
#     df_train = df_train.merge(df_dummies, how="left", left_index=True, right_index=True)
    
# print("shape after dummies: ", df_train.shape)

# Training LGB

In [9]:
import lightgbm as lgb

features_to_drop = [TARGET]
features = [ft for ft in df_train.columns if ft not in features_to_drop]

importances = df_train[features].columns.to_frame()
train_preds = df_train[TARGET].to_frame()
train_preds["preds"] = 0

df_sub["predicted"] = 0
df_sub[TARGET] = 0

kfold = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
for fold, (train_index, valid_index) in enumerate(kfold.split(df_train, df_train[TARGET])):
    x_train, x_valid = df_train.loc[train_index][features], df_train.loc[valid_index][features]
    y_train, y_valid = df_train.loc[train_index][TARGET], df_train.loc[valid_index][TARGET]
    
    clf = lgb.LGBMClassifier(
        learning_rate=0.01,
        n_estimators=5000,
        random_state=SEED,
        boosting="gbdt",
        objective="binary",
        subsample=0.8,
        subsample_freq=10,
        colsample_bytree=0.8,
        max_depth=5
    )
    
    clf.fit(
        x_train,
        y_train,
#         categorical_feature=cols,
        eval_set=[(x_valid, y_valid)],
        eval_metric="auc",
        early_stopping_rounds=500,
        verbose=False
    )
    
    y_pred = clf.predict_proba(x_valid, num_iteration=clf.best_iteration_)[:, 1]
    train_preds.loc[valid_index, "preds"] = y_pred
    
    test_preds = clf.predict_proba(df_test, num_iteration=clf.best_iteration_)[:, 1]
    df_sub[TARGET] += test_preds/FOLDS
    
    importances[fold] = clf.feature_importances_    
    
    print(f"Fold {fold+1}, AUC: {roc_auc_score(y_valid, y_pred)}")
    gc.collect()

print(f"CV AUC: {roc_auc_score(train_preds[TARGET], train_preds['preds'])}")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/mario/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-625ca737e0ae>", line 37, in <module>
    verbose=False
  File "/home/mario/venv/lib/python3.6/site-packages/lightgbm/sklearn.py", line 800, in fit
    callbacks=callbacks)
  File "/home/mario/venv/lib/python3.6/site-packages/lightgbm/sklearn.py", line 595, in fit
    callbacks=callbacks)
  File "/home/mario/venv/lib/python3.6/site-packages/lightgbm/engine.py", line 256, in train
    evaluation_result_list.extend(booster.eval_valid(feval))
  File "/home/mario/venv/lib/python3.6/site-packages/lightgbm/basic.py", line 2137, in eval_valid
    return [item for i in range_(1, self.__num_dataset)
  File "/home/mario/venv/lib/python3.6/site-packages/lightgbm/basic.py", line 2138, in <listcomp>
    for item in self.__inner_eval(self.name_valid_sets[i - 1], i, feval)]
  File

KeyboardInterrupt: 

In [ ]:
f1, precision, recall = [], [], []
max_f1_threshold = -1
max_f1 = -1
for i in range(1000):
    _i = i / 1000.0
    
    y_pred = np.where(train_preds["preds"] > _i, 1, 0)
    
    _precision = precision_score(train_preds[TARGET], y_pred)
    if _precision == 1.0:
        break
    _f1 = f1_score(train_preds[TARGET], y_pred)
    if _f1 > max_f1:
        max_f1 = _f1
        max_f1_threshold = _i
    
    f1.append(_f1)
    precision.append(_precision)
    recall.append(recall_score(train_preds[TARGET], y_pred))
    
plt.figure(figsize=[10, 5])
plt.plot(range(len(f1)), f1, label="f1")
plt.plot(range(len(f1)), precision, label="precision")
plt.plot(range(len(f1)), recall, label="recall")
plt.legend()
plt.tight_layout()
plt.show()

print(f"max_f1 [{max_f1_threshold}]: {max_f1}")

## Clip and save sub

In [ ]:
df_sub[TARGET].hist()

In [ ]:
df_sub["predicted"] = np.where(df_sub[TARGET] > max_f1_threshold, 1, 0)
df_sub.head()

In [ ]:
df_sub[["predicted", "y"]].describe()

In [ ]:
train_preds.columns = ["predicted", "y"]
train_preds["predicted"] = np.where(train_preds["y"] > max_f1_threshold, 1, 0)
train_preds.head()

In [ ]:
train_preds["id"] = df_train["id"]
train_preds = train_preds[["id", "predicted", "y"]]
train_preds.head()

In [ ]:
train_preds.to_csv(f"../output/mario/train_preds_{VERSION}.csv", index=False)
df_sub.to_csv(f"../output/mario/test_preds_{VERSION}.csv", index=False)
df_sub.drop("y", axis=1).to_csv(f"../output/mario/sub_{VERSION}.csv", index=False)

## Feature importance

In [ ]:
importances_sorted = importances.mean(axis=1).sort_values(ascending=False).head(30)
plt.figure(figsize=(10,15))
sns.barplot(y=importances_sorted.index, x=importances_sorted, orient="h")